# Add environment variable to here

In [1]:
from my_os_variables import *

In [2]:
import os

minio_url = os.environ["minio_url"]
minio_access_key = os.environ['minio_access_key']
minio_secret_key = os.environ['minio_secret_key']
minio_region = os.environ['minio_region']
minio_bucket_name = os.environ['minio_bucket_name']

competition_name = os.environ['competition_name']

kaggle_access_key = os.environ["kaggle_access_key"]
kaggle_secret_key = os.environ["kaggle_secret_key"]

pv_mount_name = os.environ['pv_mount_name']
pv_count = os.environ['pv_count']

download_from = os.environ['download_from']

In [5]:
import kfp
import kfp.dsl as dsl

from kubernetes import client as k8s_client

from istio_auth_with_client import kfp_client

{'experiments': [{'created_at': datetime.datetime(2023, 9, 9, 4, 10, 57, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'c6b640f2-b998-45ed-b856-65af8139c0fc',
                  'name': 'global_warming_level_3',
                  'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}


## pv 문제인지 pipeline 문제인지 모르겠으나 같은 이름이 들어가 있는 상태로 run이 되면 pvc가 생성이 되지 않는 버그를 발견했다.(이전pipeline을 terminate 시켜도)
## 따라서 이름 앞에 랜덤 문자열을 붙였다.

In [6]:
import string
import random

_LENGTH = 8 # 몇자리?
string_pool = string.ascii_lowercase
tmp_hash = "" # 결과 값
for i in range(_LENGTH) :
    # 랜덤한 하나의 숫자를 뽑아서, 문자열 결합을 한다.
    tmp_hash += random.choice(string_pool)
print(tmp_hash)

ojjnjlny


In [7]:
#타입 명시 안해주면 이상한 값이 들어오는듯?
def sleep_for_a_while(sleep_time : int):
    import time
    print("sleep for ", sleep_time, 'seconds')
    time.sleep(sleep_time)

In [8]:
#sleep_for_a_while(5)
#print("good")

In [9]:

@dsl.pipeline(
    name=tmp_hash+"level3",
    description=tmp_hash+"level3"
)
def global_warming_level_3_pipeline():
    
    
    #list 보다 dict가 낫나?
    volume_list = []
    volume_dict = dict()
    
    for pv_num in range(1, int(pv_count)+1):
        
        mount_path = "/var/source/" + pv_mount_name + str(pv_num)
        
        vop = dsl.VolumeOp(name=tmp_hash+"create_pvc_"+str(pv_num),
                   resource_name=tmp_hash+'my-pvc-'+str(pv_num),
                   size='1Gi',#실전에서는 크게 잡아야됨.
                   modes=dsl.VOLUME_MODE_RWO, #RWO : read write once
                   set_owner_reference = True # https://github.com/kubeflow/pipelines/issues/6649#issuecomment-938509228
                  )
        volume_list.append(mount_path)
        volume_dict[mount_path] = vop.volume
    
    #print("volume_list", volume_list)
    #print("volume_dict", volume_dict)
    
    
    mount_path = "/var/source/global_warming"
    download_pv = dsl.VolumeOp(name=tmp_hash+"create_pvc_0",
                       resource_name=tmp_hash+'my-pvc-0',
                       size='400Gi',
                       modes=dsl.VOLUME_MODE_RWO, #RWO : read write once
                       set_owner_reference = True # https://github.com/kubeflow/pipelines/issues/6649#issuecomment-938509228
                       )
    volume_list.append(mount_path)
    volume_dict[mount_path] = download_pv.volume
    
    
    step_1 = kfp.dsl.ContainerOp(
        name="download_and_extract",
        image="escs34/global_warming:level_3_12gb_v17", #command를 넣을 수 있음.. argument랑
        command=["python3"],
        arguments=[
            "/var/source/level_3_12gb_main_a.py"
        ],
        #여기서 한번에 넣으려고 했는데 그랬더니 마지막 pv만 딱 연결됨. add_pvolume으로 되나 봐야 겠음
        #=> pv_num이 아니라 pv_count로 넣어서 마지막 값만 들어감... dict에 마지막 값밖에 없었음
        pvolumes=volume_dict
        #pvolumes={'/var/source/global_warming': vop.volume}
    )
    
    
    #for key, value in volume_dict.items():
    #    print("key, value : ", key)#,value)
    #    step_1.add_pvolumes({key:value})
    

    #이거 썼더니 2번 돌리면 "어머 이전 값이 캐시에 있네요^^? 하면서 스킵해버림
    #sleep_for_a_while_op = kfp.components.func_to_container_op(sleep_for_a_while)
    #sleep_time = 10
    #sleep_for_a_while_task = sleep_for_a_while_op(sleep_time)
    
    sleep_for_scale_out = kfp.dsl.ContainerOp(
        name="download_and_extract",
        image="escs34/global_warming:level_2_v5", #command를 넣을 수 있음.. argument랑
        command=["sleep"],
        arguments=[
            "300"
        ],
        #여기서 한번에 넣으려고 했는데 그랬더니 마지막 pv만 딱 연결됨. add_pvolume으로 되나 봐야 겠음
        #=> pv_num이 아니라 pv_count로 넣어서 마지막 값만 들어감... dict에 마지막 값밖에 없었음
        pvolumes=volume_dict
        #pvolumes={'/var/source/global_warming': vop.volume}
    )
    
    
    
    #step_2 는 pv 숫자 만큼 만들어 져야 함.
    step_2_list = []
    for pv_num in range(1,int(pv_count)+1):
        mount_path = "/var/source/" + pv_mount_name + str(pv_num)
        step_2 = kfp.dsl.ContainerOp(
            name = "resize_reformat_zip_data_and_upload",
            image="escs34/global_warming:level_3_12gb_v17",
            command=["python3"],
            arguments=[
                "/var/source/level_3_12gb_main_b.py"
            ],
            #pvolumes=volume_dict
            pvolumes={'/var/source/'+pv_mount_name+str(pv_num): volume_dict[mount_path]}
        )
        step_2_list.append(step_2)
        
    #print("step_2_list : ", step_2_list)
    
    steps = [step_1] + step_2_list
    pv_count_list = [pv_count] + [str(pv_num) for pv_num in range(1, int(pv_count)+1)]
    print("pv_count_list : ",pv_count_list)
    
    for i in range(len(steps)):
        steps[i].add_env_variable(k8s_client.V1EnvVar(name='minio_url', value=minio_url)) \
            .add_env_variable(k8s_client.V1EnvVar(name='minio_access_key', value=minio_access_key)) \
            .add_env_variable(k8s_client.V1EnvVar(name='minio_secret_key', value=minio_secret_key)) \
            .add_env_variable(k8s_client.V1EnvVar(name='minio_region', value=minio_region)) \
            .add_env_variable(k8s_client.V1EnvVar(name='minio_bucket_name', value=minio_bucket_name)) \
            .add_env_variable(k8s_client.V1EnvVar(name='competition_name', value=competition_name)) \
            .add_env_variable(k8s_client.V1EnvVar(name='kaggle_access_key', value=kaggle_access_key)) \
            .add_env_variable(k8s_client.V1EnvVar(name='kaggle_secret_key', value=kaggle_secret_key)) \
            .add_env_variable(k8s_client.V1EnvVar(name='pv_mount_name', value=pv_mount_name)) \
            .add_env_variable(k8s_client.V1EnvVar(name='pv_count', value=pv_count_list[i])) \
            .add_env_variable(k8s_client.V1EnvVar(name='download_from', value=download_from))

        
        
        
    #build the pipelien with definded container
    
    sleep_for_scale_out.after(step_1)#
    
    #for pv_num in range(int(pv_count)):
    #    step_2_list[pv_num].after(step_1)
    
    for pv_num in range(int(pv_count)):
        step_2_list[pv_num].after(sleep_for_scale_out)
    
    #pod를 compelete 후 300초 뒤에 삭제 (삭제해야 pv도 삭제돼서 사용.)
    dsl.get_pipeline_conf().set_ttl_seconds_after_finished(300)

    

In [10]:
from kfp import compiler

In [11]:
exp = kfp_client.create_experiment(name="global_warming_level_3")
exp = kfp_client.get_experiment(experiment_name="global_warming_level_3")

In [12]:
compiler.Compiler().compile(global_warming_level_3_pipeline, "level_3.yaml")

/opt/conda/lib/python3.8/site-packages/kfp/dsl/_container_op.py:1261: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(


pv_count_list :  ['3', '1', '2', '3']


In [13]:
run = kfp_client.run_pipeline(exp.id, "level_3", "level_3.yaml")